In [1]:
import pandas as pd
import numpy as np
import recordlinkage 

In [2]:
amazon_train = pd.read_csv("preprocessed_amazon_train.csv", delimiter = ",")
google_train = pd.read_csv("preprocessed_google_train.csv", delimiter = ",")
amazon_test = pd.read_csv("preprocessed_amazon_test.csv", delimiter = ",")
google_test = pd.read_csv("preprocessed_google_test.csv", delimiter = ",")
train_perfect_matching = pd.read_csv("preprocessed_train_perfect_matching.csv", delimiter = ",")
test_perfect_matching = pd.read_csv("preprocessed_test_perfect_matching.csv", delimiter = ",")

In [21]:
key_indexer_1 = recordlinkage.Index()
key_indexer_1.block(left_on="name",right_on="name")
block_index_1 = key_indexer_1.index(amazon_train,google_train)

In [22]:
block_amazon_id = []
block_google_id = []
# get the amazon id from blocking
for i in range(0,len(block_index_1)):
    block_amazon_id.append(amazon_train["idAmazon"][block_index_1[i][0]])
# get the google id from blocking
for i in range(0,len(block_index_1)):
    block_google_id.append(google_train["idGoogle"][block_index_1[i][1]])  
    
block_potential_matching_1 = pd.DataFrame({"idAmazon":block_amazon_id,"idGoogle":block_google_id})
print(len(block_potential_matching_1))

119


In [23]:
# Generate the labels
def negatives_generator(perfect_matching,potential_matching):
    # check the quality of blcoking
    auxiliary = pd.merge(perfect_matching,potential_matching, on=["idAmazon","idGoogle"], how="outer", indicator=True)
    print("true positve/recall: "+str(len(*np.where(auxiliary["_merge"]=="both"))))
    print("false positive/- samples: "+str(len(*np.where(auxiliary["_merge"]=="right_only"))))
    print("false negative/+ lost: "+str(len(*np.where(auxiliary["_merge"]=="left_only")))+"\n")
    # labelling
    auxiliary["label"] = np.where(auxiliary["_merge"]=="both",1,0) 
    print("No. of positives: "+str(len(*np.where(auxiliary["label"]==1))))
    print("No. of negatives: "+str(len(*np.where(auxiliary["label"]==0)))+"\n")
    auxiliary = auxiliary[["idAmazon","idGoogle","label"]]
#     auxiliary['similarity'].fillna(1,inplace=True) 
    return auxiliary

In [28]:
# Label
train_index_labels = negatives_generator(train_perfect_matching,block_potential_matching_1)
#[similarity, idAmazon, idGoogle, label]

true positve/recall: 57
false positive/- samples: 62
false negative/+ lost: 1009

No. of positives: 57
No. of negatives: 1071



In [18]:
amazon_train["name_manu"] = amazon_train["name"]+amazon_train["manufacturer"]
google_train["name_manu"] = google_train["name"]+google_train["manufacturer"]

In [20]:
key_indexer_2 = recordlinkage.Index()
key_indexer_2.block(left_on="name_manu",right_on="name_manu")
block_index_2 = key_indexer_2.index(amazon_train,google_train)

In [26]:
block_amazon_id = []
block_google_id = []
# get the amazon id from blocking
for i in range(0,len(block_index_2)):
    block_amazon_id.append(amazon_train["idAmazon"][block_index_2[i][0]])
# get the google id from blocking
for i in range(0,len(block_index_2)):
    block_google_id.append(google_train["idGoogle"][block_index_2[i][1]])  
    
block_potential_matching_2 = pd.DataFrame({"idAmazon":block_amazon_id,"idGoogle":block_google_id})
print(len(block_potential_matching_2))

4


In [29]:
# Label
train_index_labels = negatives_generator(train_perfect_matching,block_potential_matching_2)
#[similarity, idAmazon, idGoogle, label]

true positve/recall: 4
false positive/- samples: 0
false negative/+ lost: 1062

No. of positives: 4
No. of negatives: 1062

